## Use PDF documents with RAG
This tutorial downloads a set of PDF files from the Analog Devices website. You can use your own later. We will use these documents as our RAG knowledge base, with the goal of asking the LLM questions about information they contain.

In this tutorial:
1. Download the documents
2. We will extract text from the documents and break it apart into chunks
3. Vectorize the text chunks so the vector database can search them efficiently (semantic search)
4. Load the vectors and text chunks into the database.

### Document download
We will download documents from [this page](https://www.analog.com/en/lp/001/blackfin-manuals.html) using Pandas' awesome capabilities. 

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from urllib.parse import urlparse
import os
import subprocess

url = "https://www.analog.com/en/lp/001/blackfin-manuals.html"
directory = "docs2"

try:
    # download the file
    subprocess.run(["wget", url])
    parsed_url = urlparse(url)
    file_name = os.path.basename(parsed_url.path)
    # open the file
    with open(file_name, 'r', encoding='utf-8') as file:
        file_content = file.read()
    
    # parse the webpage
    soup = BeautifulSoup(file_content, "html.parser")
    
    # find pdf links
    pdf_links = []
    for link in soup.find_all("a", href=True):
        href = link["href"]
        if href.endswith(".pdf"):
            pdf_links.append(href)

    # make directory to store the pdfs in
    os.makedirs(directory, exist_ok = True)
    
    # download pdfs
    for cur_pdf in tqdm(pdf_links, total=len(pdf_links), desc="Downloading", unit="files"):
        response = requests.get(cur_pdf)
        # get file name
        parsed_url = urlparse(cur_pdf)
        file_name = target_directory + "/" + os.path.basename(parsed_url.path)     
        with open(file_name, "wb") as file:
            file.write(response.content)

except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"An error occurred: {err}")

[0] Downloading 'https://www.analog.com/en/lp/001/blackfin-manuals.html' ...
Saving 'blackfin-manuals.html'
HTTP response 200  [https://www.analog.com/en/lp/001/blackfin-manuals.html]


Downloading: 100%|██████████████████████████| 91/91 [00:12<00:00,  7.40files/s]


### Break documents into text chunks

In [32]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

# load the documents
def load_documents():
    document_loader = PyPDFDirectoryLoader("docs2")
    return document_loader.load()

# split documents to managable chunks
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 800,
        chunk_overlap = 80,
        length_function = len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

documents = load_documents()
chunks = split_documents(documents)
print(chunks[0])




page_content='ADSP-BF7xx Blackfin+ Processor
Programming Reference
Revision 1.0, October 2016
Part Number
82-100123-01
Analog Devices, Inc.
Three T echnology Way
Norwood MA , 02062' metadata={'source': 'docs2/ADSP-BF70x_Blackfin_Programming_Reference.pdf', 'page': 0}


In [33]:
print (chunks[330])

page_content='(W32)
Input data operand format is signed fractional with no extension bits in the accumulators at 32 bits. Left-shift
correction of the product is performed, as required. This option is used for legacy GSM speech vocoder algo-
rithms written for 32-bit accumulators. For this option only, this special case applies:
0x8000 x 0x8000 = 0x7FFF
(M)
Operation uses mixed-multiply mode. Valid only for MAC1 versions of the instruction. Multiplies a signed
fractional operand by an unsigned fractional operand with no left-shift correction, where the first operand is
signed and the second is unsigned. MAC0 performs an unmixed multiplication of signed fractional operands
unless another format as specified (i.e., MAC0 executes the specified signed/signed or unsigned/unsigned mul-' metadata={'source': 'docs2/ADSP-BF70x_Blackfin_Programming_Reference.pdf', 'page': 78}


Let's check how many chunks were generated from our documents...

In [34]:
len(chunks)

68386

In [35]:
print(chunks[330].page_content)
print(chunks[330].metadata['source'])
print(chunks[330].metadata['page'])

(W32)
Input data operand format is signed fractional with no extension bits in the accumulators at 32 bits. Left-shift
correction of the product is performed, as required. This option is used for legacy GSM speech vocoder algo-
rithms written for 32-bit accumulators. For this option only, this special case applies:
0x8000 x 0x8000 = 0x7FFF
(M)
Operation uses mixed-multiply mode. Valid only for MAC1 versions of the instruction. Multiplies a signed
fractional operand by an unsigned fractional operand with no left-shift correction, where the first operand is
signed and the second is unsigned. MAC0 performs an unmixed multiplication of signed fractional operands
unless another format as specified (i.e., MAC0 executes the specified signed/signed or unsigned/unsigned mul-
docs2/ADSP-BF70x_Blackfin_Programming_Reference.pdf
78


Let's save the chunks in case we need them again...

In [36]:
import pickle

with open("docs2/text_chunks.pkl", "wb") as file:  # 'wb' means write in binary mode
    pickle.dump(chunks, file)

### Embedding the chunks
Now we need to embed the chunks in the database.

As you can see, each chunk consists of [Langchain Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document). 
Each Document has two elements:
* ```page_content``` - the text of the chunk
* ```metadata``` - a dictionary object, which in our case, contains the source document path and the page in which the chunk appeared.

We will need to store this information in our vector database, Weaviate. Therefore, we will need to start with a very simple schema.
* chunk_content
* chunk_document_name
* chunk_document_page

Weaviate will handle the embedding for us using the model we specified (OpenAI).

In [7]:
import weaviate.classes.config as wc
import weaviate
import os

headers = {
    "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")
}  # Replace with your OpenAI API key

client = weaviate.connect_to_local(headers=headers)

client.collections.create(
    name="ADI_DOCS",
    properties=[
        wc.Property(name="chunk_content", data_type=wc.DataType.TEXT),
        wc.Property(name="chunk_document_name", data_type=wc.DataType.TEXT),
        wc.Property(name="chunk_document_page", data_type=wc.DataType.INT),
    ],
    # Define the vectorizer module
    vectorizer_config=wc.Configure.Vectorizer.text2vec_openai(),
    # Define the generative module
    generative_config=wc.Configure.Generative.openai()
)

client.close()

/home/yuvalzukerman/.local/lib/python3.12/site-packages/weaviate/warnings.py:312: UserWarning: Bat005: Rate limit reached with error WeaviateInsertManyAllFailedError('Every object failed during insertion. Here is the set of all errors: connection to: OpenAI API failed with status: 429 request-id: req_94b0657936662af5b43d0957fcf3ed0f error: Rate limit reached for text-embedding-ada-002 in organization org-TpPP0kLjXaLunR4pFEPauR5c on tokens per min (TPM): Limit 1000000, Used 996519, Requested 88128. Please try again in 5.078s. Visit https://platform.openai.com/account/rate-limits to learn more.').
            Sleeping for 62 seconds.
  warnings.warn(
/home/yuvalzukerman/.local/lib/python3.12/site-packages/weaviate/warnings.py:228: DeprecationWarning: Dep020: The `all_responses` attribute in the `BatchResults` object is deprecated and will be removed by Q4 2024. Please instead use the `errors` and `uuids` attributes.
  warnings.warn(
/home/yuvalzukerman/.local/lib/python3.12/site-packages

#### Load Weaviate with the data!

In [24]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

try:    
    # connect to database
    client = weaviate.connect_to_local(headers=headers)
           
    # Get the collection
    adi_docs = client.collections.get("ADI_DOCS")
    
    cur_page = 0
    cur_doc = ""
    i = 0
    
    # Enter context manager
    with adi_docs.batch.dynamic() as batch:
        # Loop through the data
        for chunk in tqdm(chunks, total=len(chunks)):
            i +=1
            # Convert data types

            chunk_obj = {
                "chunk_content": chunk.page_content,
                "chunk_document_name": chunk.metadata['source'],
                "chunk_document_page": chunk.metadata['page'],
            }

            if cur_doc != chunk.metadata['source']:
                cur_doc = chunk.metadata['source']

            if cur_page != chunk.metadata['page']:
                cur_page = chunk.metadata['page']
                
            seed = cur_doc + ":" + str(cur_page) + ":" + str(i)
    
            # Add object to batch queue
            batch.add_object(
                properties=chunk_obj,
                uuid=generate_uuid5(seed)
                # references=reference_obj  # You can add references here
            )
            # Batcher automatically sends batches
    
    # Check for failed objects
    if len(adi_docs.batch.failed_objects) > 0:
        print(f"Failed to import {len(adi_docs.batch.failed_objects)} objects")
finally:
    client.close()

100%|███████████████████████████████████| 68090/68090 [10:48<00:00, 104.92it/s]


Let's verify that the records are in the vector database:

In [25]:
try:    
    # connect to database
    client = weaviate.connect_to_local(headers=headers)
           
    # Get the collection
    adi_docs = client.collections.get("ADI_DOCS")
    response = adi_docs.aggregate.over_all(total_count=True)
    print(response.total_count)

finally:
    client.close()

68090
